In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def readData(name):
    train = pd.read_csv(name+".csv")
    train=train.drop(["date"], axis=1)
    train=train.drop(["Unnamed: 0"], axis=1)
    return train

def sta(train):
    train=train.convert_objects(convert_numeric=True)
    train= train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return train

def buildTrain(train, pastDay, futureDay):
    X_train, Y_train ,Z_train= [], [],[]
    for i in range(train.shape[0]-futureDay-pastDay+1):
        X_train.append(np.array(train.iloc[i:i+pastDay]))
        a=np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["high"])
        b=np.array(train.iloc[i+pastDay-1:i+pastDay+futureDay-1]["high"])
        if a>=b:
            Y_train.append(1)
        else:
            Y_train.append(0)                                                       
    return np.array(X_train), np.array(Y_train)

def shuffle(X,Y):
    np.random.seed()
    randomList = np.arange(X.shape[0])
    np.random.shuffle(randomList)
    return X[randomList], Y[randomList]

def splitData(X,Y,rate):
    X_train = X[int(X.shape[0]*rate):]
    Y_train = Y[int(Y.shape[0]*rate):]
    X_val = X[:int(X.shape[0]*rate)]
    Y_val = Y[:int(Y.shape[0]*rate)]
    return X_train, Y_train, X_val, Y_val

def buildModel(train_x,train_y,bs):
    model = Sequential()
    model.add(LSTM(32, input_length=train_x.shape[1],input_dim= train_x.shape[2],return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(1,kernel_initializer='uniform',activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    model.summary()
    callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
    model.fit(train_x,train_y, epochs=1000, batch_size=bs, validation_split=0.1, callbacks=[callback])
    return model,[32,32,0,0,0]



In [ ]:
lookback=7
batch_size=32
col_name=["1","2","3","4","5","acc","lookback","batch_size","name","num"]
place_name=["train+oil+weather+國定假日+拜拜日 雲林","train+oil+weather+國定假日+拜拜日 嘉義","train+oil+weather+國定假日+拜拜日 彰化","train+oil+weather+國定假日+拜拜日 台南","train+oil+weather+國定假日+拜拜日 高雄","train+oil+weather+國定假日+拜拜日 屏東","train+oil+weather+國定假日+拜拜日 台中","train+oil+weather+國定假日+拜拜日 苗栗","train+oil+weather+國定假日+拜拜日 桃園","train+oil+weather+國定假日+拜拜日 台北"]
for i in place_name:
    df=pd.DataFrame(columns=col_name)
    data=[]
    for j in range(30):
        train=readData(i)
        temp=train
        train=sta(train)
        train_x1,train_y1=buildTrain(train,lookback,1)
        train_x2,train_y2=buildTrain(temp,lookback,1)
        train_x,train_y=train_x1,train_y2
        train_x,train_y= shuffle(train_x,train_y)
        train_x,train_y, val_x, val_y= splitData(train_x,train_y, 0.05)
        model,layer=buildModel(train_x,train_y,batch_size)
        pre=model.evaluate(val_x, val_y)
        data.append({"1":layer[0],"2":layer[1],"3":layer[2],"4":layer[3],"5":layer[4],"acc":pre,"lookback":lookback,"batch_size":batch_size,"name":i,"num":j})
        print(pre)
    df=pd.concat([pd.DataFrame(data), df], ignore_index=True,sort=True)
    df.to_csv('class'+i+'_data'+'(lookback  '+str(lookback)+')'+'(bs  '+str(batch_size)+')'+'.csv', encoding='utf_8_sig')

C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:39: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:39: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(32, return_sequences=True, input_shape=(7, 27))`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 7, 32)             7680      
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 16,033
Trainable params: 16,033
Non-trainable params: 0
_________________________________________________________________
Train on 2257 samples, validate on 251 samples
Epoch 1/1
131/131 [==============================] - 0s 61us/step
0.6895623875938299
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 7, 32)             7680      
_________________________________________________________________
lstm_4

In [14]:
train=readData("train+oil+weather+國定假日+拜拜日 雲林")
a=train[train['volume']==124858.0].index.item()
train[30:60]

,最低氣溫(℃),最大陣風(m/s),最大陣風風向(360degree),最小相對溼度(%),最高氣溫(℃),氣溫(℃),測站最低氣壓(hPa),測站最高氣壓(hPa),相對溼度(%),風向(360degree),...,low,mean,medium,volume,無鉛汽油92,無鉛汽油95,無鉛汽油98,超級/高級柴油,國定假日,拜拜日
30,10.6,3.2,110,35,19.6,14.6,889.2,892.6,58,118,...,0.0,0.0,0.0,0.0,17.2,18.2,19.2,13.4,0.0,1.0
31,9.4,4.1,282,43,17.4,13.0,888.6,891.9,78,314,...,18.7,24.2,24.4,170213.0,17.2,18.2,19.2,13.4,0.0,0.0
32,10.9,5.4,340,29,21.5,14.1,890.1,893.2,74,155,...,15.2,20.2,19.7,126482.0,17.2,18.2,19.2,13.4,0.0,0.0
33,10.7,6.1,127,59,16.2,12.6,890.3,893.0,85,115,...,18.4,23.2,23.3,106902.0,17.2,18.2,19.2,13.4,0.0,0.0
34,10.2,2.8,352,45,21.5,15.4,890.0,893.5,75,110,...,18.3,22.3,22.3,146239.0,17.2,18.2,19.2,13.4,1.0,0.0
35,13.0,4.1,108,36,22.5,16.8,888.5,891.9,74,321,...,22.3,25.2,25.3,145622.0,17.2,18.2,19.2,13.4,1.0,0.0
36,14.6,12.6,181,43,24.3,19.3,884.4,888.8,64,153,...,25.6,28.4,28.6,153983.0,17.2,18.2,19.2,28.7,0.0,1.0
37,12.8,5.7,276,56,20.5,16.1,885.0,888.5,84,220,...,0.0,0.0,0.0,0.0,17.2,18.2,19.2,28.7,0.0,0.0
38,10.3,6.4,232,90,14.3,11.9,887.9,891.3,91,266,...,0.0,0.0,0.0,0.0,17.2,18.2,19.2,28.7,0.0,0.0
39,10.7,3.9,125,61,16.6,12.8,889.1,891.3,85,321,...,20.4,23.3,23.1,187390.0,17.2,18.2,19.2,28.7,0.0,0.0
